In [ ]:
import pandas as pd

# Read all sheets
excel_path = "IS453 Group Assignment - Data Dict.xlsx"
xls = pd.ExcelFile(excel_path)

# Display sheet names
print(xls.sheet_names)

# Read each sheet into a DataFrame
df1 = pd.read_excel(xls, xls.sheet_names[0])
df2 = pd.read_excel(xls, xls.sheet_names[1])

# Display basic info
print("Sheet 1 shape:", df1.shape)
print("Sheet 2 shape:", df2.shape)

In [ ]:
import pandas as pd

# This was after preprocessing of removing applicant below 60 years
main_df = pd.read_csv("elder_care_segment.csv")

In [ ]:
main_df["NAME_INCOME_TYPE"].unique()